# 轻量级时序分析 Agent Demo

基于 **Prophet + DeepSeek + Pydantic AI** 的简洁实现

## 依赖
```bash
pip install prophet pydantic-ai akshare pandas matplotlib
```

In [ ]:
# 环境配置
import nest_asyncio
nest_asyncio.apply()

import os
os.environ["DEEPSEEK_API_KEY"] = "your-deepseek-api-key"  # 替换为你的 key

In [ ]:
# 导入我们的轻量级 Agent
from timeseries_agent import TimeSeriesPipeline, quick_forecast, plot_forecast
import akshare as ak
import pandas as pd

## 方式1: 快速预测 (一行代码)

In [ ]:
# 获取数据
df = ak.stock_zh_a_hist(
    symbol="000001",  # 平安银行
    period="daily",
    start_date="20240101",
    end_date="20241231",
    adjust=""
)
print(f"获取到 {len(df)} 条数据")
df.head()

In [ ]:
# 一行代码预测
result = quick_forecast(
    df, 
    horizon=30, 
    query="分析平安银行走势，预测未来30天价格趋势，给出投资建议"
)

In [ ]:
# 查看分析报告
print(result["analysis"])

In [ ]:
# 查看预测值
print("预测结果 (前10天):")
for item in result["forecast"][:10]:
    print(f"  {item['date']}: {item['value']:.2f} [{item['lower']:.2f}, {item['upper']:.2f}]")

## 方式2: 分步执行 (更多控制)

In [ ]:
# 初始化管道
pipeline = TimeSeriesPipeline()

# 准备数据
prepared_df = pipeline.prepare_data(df, date_col="日期", value_col="收盘")
prepared_df.head()

In [ ]:
# 分析特征
from timeseries_agent import FeatureAnalyzer

features = FeatureAnalyzer.analyze(prepared_df)
print(f"趋势: {features.trend}")
print(f"波动性: {features.volatility}")
print(f"季节性: {features.seasonality}")
print(f"摘要: {features.summary}")

In [ ]:
# 执行预测 (不带 LLM 分析，更快)
result_no_llm = pipeline.forecast(
    prepared_df, 
    horizon=30, 
    model="prophet",
    query=None  # 不调用 LLM
)

In [ ]:
# 可视化
plot_forecast(prepared_df, result_no_llm["forecast"], title="平安银行 30天预测")

## 方式3: 指数预测

In [ ]:
# 获取上证指数
index_df = ak.stock_zh_index_daily_em(symbol="sh000001")
index_df = index_df.tail(365)  # 最近一年
print(f"获取到 {len(index_df)} 条数据")
index_df.head()

In [ ]:
# 预测上证指数
result_index = quick_forecast(
    index_df,
    horizon=20,
    date_col="date",
    value_col="close",
    query="分析上证指数走势，预测未来20天"
)

print(result_index["analysis"])

## 架构说明

```
┌─────────────────────────────────────────────────────┐
│                  TimeSeriesPipeline                 │
├─────────────────────────────────────────────────────┤
│  1. prepare_data()     - 数据标准化 (ds, y)         │
│  2. FeatureAnalyzer    - 特征分析 (趋势/波动/季节)   │
│  3. ModelDispatcher    - 模型调度 (Prophet/...)     │
│  4. DeepSeek LLM       - 结果解释 + 投资建议        │
└─────────────────────────────────────────────────────┘
```

### 后续扩展
- 添加 ARIMA, ETS 等传统模型
- 集成 Chronos, TimesFM 等 AI 模型
- 添加交叉验证和模型选择
- 支持多变量预测